# Chi Analysis with Resampling


---
**License**

Chi Analysis with Resampling

 Fri Mar 14 08:50:00 2025\
 Copyright  2025\
 Cinthia Nunes de Lima <cinthia.n.lima@uel.br> \ Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 Chi_Analysis_Resample \
 Copyright (C) 2024 Cinthia Nunes de Lima <cinthia.n.lima@uel.br>, Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

## Packages & Configs

In [1]:
import sys
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/Scripts")

#NumCosmo
from numcosmo_py import Ncm, Nc, GObject
from richness_mass_calib import create_richness_mass_calib

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

#Useful packages
import numpy as np
import math
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# GCRCatalogs
sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")
import GCRCatalogs
GCRCatalogs.set_root_dir_by_site("nersc")

#Calibration functions
from bdata import DataB
from model_fitting import ModelFitting

## Data

### Clevar

In [2]:
# CLEVAR

# min_richness = 5
# min_halo_mass = 1e13 #Msun

# Halos/Clusters
# #CosmoDC2 + RedMaPPer: 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
# RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

# #CosmoDC2: 'cosmoDC2_v1.1.4_image'
# halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

# #Members
# RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

# #Match catalogs
# mt = MembershipMatch()

# mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/cosmoDC2_RM_match_cross_membership')


### Data matched by ID

In [3]:
# test_matchdata = Table.read('CatalogsMatching/match_ID.fits')

# rich_data = test_matchdata["richness"]
# z_data = test_matchdata["redshift"]
# mass_data = test_matchdata["halo_mass"]
# r_err_data = test_matchdata["richness_err"]

# test_dt = Table([mass_data, rich_data, z_data, r_err_data], names=('mass', 'richness', 'redshift', 'richness_err'))


### NumCosmo's Matching By Proximity

In [4]:
mdata = Table.read('CatalogsMatching/match_proximity.fits')

rich_data = mdata["R"]
z_data = mdata["z"]
mass_data = mdata["mass"]

test_dt = Table([mass_data, rich_data, z_data], names=('mass', 'richness', 'redshift'))


### Data threshold

In [5]:
test_dt = test_dt[test_dt['mass'] > 1e13]
test_dt = test_dt[np.logical_and(test_dt['redshift'] > 0, test_dt['redshift']< 3)]
test_dt = test_dt[np.logical_and(test_dt['richness'] > 0, test_dt['richness'] < 1e10)]

mass_dt = test_dt['mass']
red_dt = test_dt['redshift']
rich_dt = test_dt['richness']

print('Data')
print('\t\tMax \t\t\tMin')
print(f'Mass:\t\t{max(mass_dt):.2e}, \t\t{min(mass_dt):.2e}')
print(f'Redshift:\t{max(red_dt)}, \t{min(red_dt)}')
print(f'Richness:\t{max(rich_dt)}, \t{min(rich_dt)}')



Data
		Max 			Min
Mass:		1.90e+15, 		1.00e+13
Redshift:	1.1764284278635397, 	0.09084865138884513
Richness:	266.2950439453125, 	5.007844924926758


### Binning

In [6]:
# #Length of bins
# zl = 0.05
# ml = 0.05
# bd = DataB(test_dt, zl, ml)
# binned_mean = bd.get_bins_mean()
# binned_std = bd.get_bins_std()

# lnM_binned_mean = np.log(binned_mean['mass'])
# z_binned_mean = binned_mean['redshift']
# lnR_binned_mean = np.log(binned_mean['richness'])

# print(f'Total bins in binned_mean: {len(bd.get_bins_mean())}')


## Models fitting

### Resampling

In [7]:
#Resampling

def resample(dmr, cluster_m):

    area = 439.78987
    cosmo = Nc.HICosmoDEXcdm()
    cosmo.omega_x2omega_k()
    cosmo.param_set_by_name("H0", 71.0)
    cosmo.param_set_by_name("Omegab", 0.0448)
    cosmo.param_set_by_name("Omegac", 0.2648)
    cosmo.param_set_by_name("Omegak", 0.00)
    cosmo.param_set_by_name("w", -1.0)
       
    cluster_m.param_set_by_name("cut", np.log(5))

    # print(cluster_m.param_get_by_name("mup0"))

    cluster_m.param_set_by_name("mup0", cluster_m.param_get_by_name("mup0"))
    cluster_m.param_set_by_name("mup1", cluster_m.param_get_by_name("mup1"))
    cluster_m.param_set_by_name("mup2", cluster_m.param_get_by_name("mup2"))
    cluster_m.param_set_by_name("sigmap0", cluster_m.param_get_by_name("sigmap0"))
    cluster_m.param_set_by_name("sigmap1", cluster_m.param_get_by_name("sigmap1"))
    cluster_m.param_set_by_name("sigmap2", cluster_m.param_get_by_name("sigmap2"))
    
    cluster_z = Nc.ClusterRedshiftNodist (z_min = 0, z_max = 1.2)
    
    mset = Ncm.MSet.new_array([cosmo, cluster_m, cluster_z])
    mset.prepare_fparam_map()

    # mset.pretty_log()

    print(mset["NcClusterMass"]["mup0"])
    
    rng = Ncm.RNG.seeded_new(None, 40)
    
    dmr.resample(mset, rng)
    
    R_resample = np.exp( dmr.props.lnR_cluster.dup_array() )
    M_resample = np.exp( dmr.props.lnM_cluster.dup_array() )
    z_resample = dmr.props.z_cluster.dup_array()
    
    return Table([M_resample, R_resample, z_resample], names=('mass', 'richness', 'redshift'))


catalog_len = len(test_dt)

lnM_v = Ncm.Vector.new(catalog_len)
z_v = Ncm.Vector.new(catalog_len)
rich_v = Ncm.Vector.new(catalog_len)

for i, mass in enumerate(test_dt['mass']):
    lnM_v.set(i, np.log(mass))

for i, z in enumerate(test_dt['redshift']):
    z_v.set(i, z)

for i, rich in enumerate(test_dt['richness']):
    rich_v.set(i, np.log(rich))

dmr = Nc.DataClusterMassRich.new()
dmr.set_data(lnM_v, z_v, rich_v)


In [8]:
fm = ModelFitting(test_dt) #Fit object

### Ascaso Fit Params

In [9]:
ascaso, fit_ascaso = fm.run_fit('ascaso')

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[09000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  4.13105590509304    [FREE]
#   -    mup1[01]:  1.13308187730147    [FREE]
#   -    mup2[02]:  0.306956374740925   [FREE]
#   - sigmap0[03]:  0.458272586294436   [FREE]
#   - sigmap1[04]: -0.0720333832727466  [FREE]
#   - sigmap2[05]: -0.0777155254988554  [FREE]
#   -     cut[06]:  1.6094379124341     [FIXED]
#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
#.........................
#  Minimum found with precisio

In [10]:
m_ascaso = Ncm.MSet.new_array([ascaso])
m_ascaso.pretty_log()

#----------------------------------------------------------------------------------
# Model[09000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  4.13105590509304    [FIXED]
#   -    mup1[01]:  1.13308187730147    [FIXED]
#   -    mup2[02]:  0.306956374740925   [FIXED]
#   - sigmap0[03]:  0.458272586294436   [FIXED]
#   - sigmap1[04]: -0.0720333832727466  [FIXED]
#   - sigmap2[05]: -0.0777155254988554  [FIXED]
#   -     cut[06]:  1.6094379124341     [FIXED]


In [12]:
area = 439.78987
cosmo = Nc.HICosmoDEXcdm()
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("H0", 71.0)
cosmo.param_set_by_name("Omegab", 0.0448)
cosmo.param_set_by_name("Omegac", 0.2648)
cosmo.param_set_by_name("Omegak", 0.00)
cosmo.param_set_by_name("w", -1.0)

ascaso.param_set_by_name("cut", np.log(5))
ascaso.param_set_by_name("mup0", ascaso.param_get_by_name("mup0"))
ascaso.param_set_by_name("mup1", ascaso.param_get_by_name("mup1"))
ascaso.param_set_by_name("mup2", ascaso.param_get_by_name("mup2"))
ascaso.param_set_by_name("sigmap0", ascaso.param_get_by_name("sigmap0"))
ascaso.param_set_by_name("sigmap1", ascaso.param_get_by_name("sigmap1"))
ascaso.param_set_by_name("sigmap2", ascaso.param_get_by_name("sigmap2"))
    
cluster_z = Nc.ClusterRedshiftNodist (z_min = 0, z_max = 1.2)
    
mset = Ncm.MSet.new_array([cosmo, ascaso, cluster_z])
mset.prepare_fparam_map()

mset.pretty_log()

#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -      H0[00]:  71                  [FIXED]
#   -  Omegac[01]:  0.2648              [FIXED]
#   -  Omegak[02]:  0                   [FIXED]
#   - Tgamma0[03]:  2.7245              [FIXED]
#   -      Yp[04]:  0.24                [FIXED]
#   -    ENnu[05]:  3.046               [FIXED]
#   -  Omegab[06]:  0.0448              [FIXED]
#   -       w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[09000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  4.13105590509304    [FREE]
#   -    mup1[01]:  1.13308187730147    [FREE]
#   -    mup2[02]

In [13]:
rng = Ncm.RNG.seeded_new(None, 40)

lnM_v = Ncm.Vector.new(len(test_dt))
z_v = Ncm.Vector.new(len(test_dt))
rich_v = Ncm.Vector.new(len(test_dt))

for i, mass in enumerate(test_dt['mass']):
    lnM_v.set(i, np.log(mass))

for i, z in enumerate(test_dt['redshift']):
    z_v.set(i, z)

for i, rich in enumerate(test_dt['richness']):
    rich_v.set(i, np.log(rich))

dmr = Nc.DataClusterMassRich.new()
dmr.set_data(lnM_v, z_v, rich_v)
dmr.resample(mset, rng)
    
R_resample = np.exp( dmr.props.lnR_cluster.dup_array() )
M_resample = np.exp( dmr.props.lnM_cluster.dup_array() )
z_resample = dmr.props.z_cluster.dup_array()

ascaso_r_data = Table([M_resample, R_resample, z_resample], names=('mass', 'richness', 'redshift'))

In [14]:
fm_resample = ModelFitting(ascaso_r_data)
ascaso_r = fm_resample.run_fit('ascaso')

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterMassRich
#----------------------------------------------------------------------------------
# Model[09000]:
#   - NcClusterMass : Ascaso Ln-normal richness distribution
#----------------------------------------------------------------------------------
# Model parameters
#   -    mup0[00]:  4.13105590509304    [FREE]
#   -    mup1[01]:  1.13308187730147    [FREE]
#   -    mup2[02]:  0.306956374740925   [FREE]
#   - sigmap0[03]:  0.458272586294436   [FREE]
#   - sigmap1[04]: -0.0720333832727466  [FREE]
#   - sigmap2[05]: -0.0777155254988554  [FREE]
#   -     cut[06]:  1.6094379124341     [FIXED]
#----------------------------------------------------------------------------------
# Model fitting. Interating using:
#  - solver:            NLOpt:ln-neldermead
#  - differentiation:   Numerical differentiantion (central)
#........................................................

In [ ]:
# ascaso_r_data = resample(dmr, ascaso)


# model.param_set_by_name("cut", np.log(5))
# mset = Ncm.MSet()
# mset.set(model)
# rmdata.m2lnL_val(mset)  
# mset.param_set_all_ftype(Ncm.ParamType.FREE) #All parameters free
    
# lh = Ncm.Likelihood.new(dmr)


# ascaso_r_data


plt.hist(np.log(test_dt['richness']), )
plt.hist(np.log(ascaso_r_data['richness']), color = 'k')

plt.show()

In [ ]:
fm_resample = ModelFitting(ascaso_r_data)
ascaso_r = fm_resample.run_fit('ascaso')


In [ ]:
lnR_mean_ascaso = fm.get_mean_model(ascaso, np.log(test_dt['mass']),  test_dt['redshift'])
lnR_std_ascaso = fm.get_std_model(ascaso, np.log(test_dt['mass']), test_dt['redshift'])

lnR_mean_ascaso_r = fm_resample.get_mean_model(ascaso_r, np.log(ascaso_r_data['mass']),  ascaso_r_data['redshift'])
lnR_std_ascaso_r = fm_resample.get_std_model(ascaso_r, np.log(ascaso_r_data['mass']), ascaso_r_data['redshift'])


### Ext (ln(1+z)) Fit Params

In [ ]:
ext_ln1pz, fit_ext = fm.run_fit('ext_ln1pz')

In [ ]:
ext_ln1pz_r = Nc.ClusterMassLnrichExt(use_ln1pz = True, lnRichness_min=0, lnRichness_max=6)

params = ["mu", "muM1", "muZ1", "muM2", "muZ2", "muMZ", "sigma0", "sigmaM1", "sigmaZ1", "sigmaM2", "sigmaZ2", "sigmaMZ"]
for p in params:
    ext_ln1pz_r.param_set_by_name(p, ext_ln1pz.param_get_by_name(p))

ext_ln1pz_r.param_set_by_name("cut", np.log(5))

mset_ext_ln1pz_r = Ncm.MSet()
mset_ext_ln1pz_r.set(ext_ln1pz_r)

mset_ext_ln1pz_r.pretty_log()

In [ ]:
ext_ln1pz_r_data = resample(dmr, ext_ln1pz_r)

In [ ]:
fm2 = FittingModel(test_dt_resampled)

In [ ]:
ext_ln1pz_r, fit_ext_r = fm2.run_fit('ext_ln1pz')

In [ ]:
lnR_mean_ext_ln1pz = fm.get_mean_model(ext_ln1pz,  np.log(test_dt['mass']), test_dt['redshift'])
lnR_std_ext_ln1pz = fm.get_std_model(ext_ln1pz,  np.log(test_dt['mass']), test_dt['redshift'])

lnR_mean_ext_ln1pz_r = fm2.get_mean_model(ext_ln1pz_r,  np.log(test_dt_resampled['mass']), test_dt_resampled['redshift'])
lnR_std_ext_ln1pz_r = fm2.get_std_model(ext_ln1pz_r,  np.log(test_dt_resampled['mass']), test_dt_resampled['redshift'])


## Errors

In [ ]:
residuals_ext = (np.log(rich_dt) - lnR_mean_ext_ln1pz) / lnR_std_ext_ln1pz
residuals_asc = (np.log(rich_dt) - lnR_mean_ascaso) / lnR_std_ascaso

residuals_ext_r = (np.log(rich_dt) - lnR_mean_ext_ln1pz) / lnR_std_ext_ln1pz
residuals_asc_r = (np.log(rich_dt) - lnR_mean_ascaso) / lnR_std_ascaso

In [ ]:
plt.figure(figsize=(8,6))

p2 = plt.scatter(rich_dt, residuals_ext, c= np.log10(mass_dt), s=5.0, label = 'Quadratic')

plt.colorbar(p2)

plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Quadratic} }{\sigma_{Quadratic \ i}}$', fontsize=15)
plt.axvline(x=10, color='g', ls='--', label='10')
plt.axvline(x=30, color='r', ls='--', label='30')
plt.axhline(y=0.0, color='k', ls='--')


plt.xscale('log')
plt.legend()
plt.xlabel('Richness', fontsize=14)

plt.show()

plt.figure(figsize=(8,6))

p1 = plt.scatter(rich_dt, residuals_asc, c= np.log10(mass_dt), s=5.0, label = 'Linear')

plt.colorbar(p1)

plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Linear} }{\sigma_{Linear \ i}}$', fontsize=15)
plt.axvline(x=10, color='g', ls='--', label='10')
plt.axvline(x=30, color='r', ls='--', label='30')
plt.axhline(y=0.0, color='k', ls='--')


plt.xscale('log')
plt.legend()
plt.xlabel('Richness', fontsize=14)

plt.show()

plt.hist2d(np.log(rich_dt), np.log(mass_dt))
plt.show()


In [ ]:
plt.figure(figsize=(8,6))

p2 = plt.scatter(test_dt_resampled['richness'], residuals_ext_r, c= np.log10(test_dt_resampled['mass']), s=5.0, label = 'Quadratic')

plt.colorbar(p2)

plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Quadratic} }{\sigma_{Quadratic \ i}}$', fontsize=15)
plt.axvline(x=10, color='g', ls='--', label='10')
plt.axvline(x=30, color='r', ls='--', label='30')
plt.axhline(y=0.0, color='k', ls='--')


plt.xscale('log')
plt.legend()
plt.xlabel('Richness', fontsize=14)

plt.show()

plt.figure(figsize=(8,6))

p1 = plt.scatter(test_dt_resampled['richness'], residuals_asc_r, c= np.log10(test_dt_resampled['mass']), s=5.0, label = 'Linear')

plt.colorbar(p1)

plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Linear} }{\sigma_{Linear \ i}}$', fontsize=15)
plt.axvline(x=10, color='g', ls='--', label='10')
plt.axvline(x=30, color='r', ls='--', label='30')
plt.axhline(y=0.0, color='k', ls='--')


plt.xscale('log')
plt.legend()
plt.xlabel('Richness', fontsize=14)

plt.show()

plt.hist2d(np.log(rich_dt), np.log(mass_dt))
plt.show()


In [ ]:
plt.figure(figsize=(8,6))

plt.scatter(rich_dt, residuals_asc, s=5.0, label = 'Linear')
plt.scatter(rich_dt, residuals_ext, s=5.0, label = 'Quadratic')

plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Quadratic} }{\sigma_{Quadratic \ i}}$', fontsize=15)
plt.axvline(x=10, color='g', ls='--', label='10')
plt.axvline(x=30, color='r', ls='--', label='30')
plt.axhline(y=0.0, color='k', ls='--')


plt.xscale('log')
plt.legend()
plt.xlabel('Richness', fontsize=14)

plt.show()

In [ ]:
plt.figure(figsize=(8,6))

p3 = plt.scatter(np.log(test_dt_resampled['mass']), residuals_asc_r, c = np.log(test_dt_resampled['richness']),  s=5.0, label = 'Linear')
plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Linear} }{\sigma_{Linear \ i}}$', fontsize=15)
plt.axvline(x=np.log(2e13), color='g', ls='--', label='2e13')
plt.axvline(x=np.log(1e14), color='r', ls='--', label='1e14')
plt.axhline(y=0.0, color='k', ls='-')

plt.colorbar(p3, label = 'lnR')

plt.legend()
plt.xlabel('ln(M)', fontsize=14)


plt.show()

plt.figure(figsize=(8,6))
p4=plt.scatter(np.log(test_dt_resampled['mass']), residuals_ext_r, c = np.log(test_dt_resampled['richness']),  s=5.0, label = 'Quadratic')
plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Linear} }{\sigma_{Linear \ i}}$', fontsize=15)
plt.axvline(x=np.log(2e13), color='g', ls='--', label='2e13')
plt.axvline(x=np.log(1e14), color='r', ls='--', label='1e14')
plt.axhline(y=0.0, color='k', ls='-')

plt.colorbar(p4, label='ln(R)')

plt.legend()
plt.xlabel('ln(M)', fontsize=14)


plt.show()

In [ ]:
plt.figure(figsize=(8,6))

p3 = plt.scatter(np.log(mass_dt), residuals_asc, c = np.log(rich_dt),  s=5.0, label = 'Linear')
plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Linear} }{\sigma_{Linear \ i}}$', fontsize=15)
plt.axvline(x=np.log(2e13), color='g', ls='--', label='2e13')
plt.axvline(x=np.log(1e14), color='r', ls='--', label='1e14')
plt.axhline(y=0.0, color='k', ls='-')

plt.colorbar(p3, label = 'lnR')

plt.legend()
plt.xlabel('ln(M)', fontsize=14)


plt.show()

plt.figure(figsize=(8,6))
p4=plt.scatter(np.log(mass_dt), residuals_ext, c = np.log(rich_dt),  s=5.0, label = 'Quadratic')
plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Linear} }{\sigma_{Linear \ i}}$', fontsize=15)
plt.axvline(x=np.log(2e13), color='g', ls='--', label='2e13')
plt.axvline(x=np.log(1e14), color='r', ls='--', label='1e14')
plt.axhline(y=0.0, color='k', ls='-')

plt.colorbar(p4, label='ln(R)')

plt.legend()
plt.xlabel('ln(M)', fontsize=14)


plt.show()

In [ ]:
# plt.scatter(np.log(mass_dt), residuals_ext, s=3.0)
# plt.ylabel(r'$\frac{\ln(R)_{i}  - < \ln(R)_{i} | M, z >_{Linear} }{\sigma_{Linear \ i}}$', fontsize=15)
# # plt.yscale('log')
# plt.xlabel('ln(M)', fontsize=14)
# plt.show()



In [ ]:

# plt.hist( test_dt_resampled["richness"] , bins=100)
# plt.show()

In [ ]:
# plt.scatter(np.log(mass_dt), residuals_ext - residuals_asc, s=3.0)
# plt.ylabel(r'$\chi_{Quad} - \chi_{Linear}$', fontsize=15)
# plt.xlabel('ln(M)', fontsize=14)
# # plt.yscale('log')
# plt.show()